Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# AutoML 07: Exploring previous runs

In this example we present some examples on navigating previously executed runs. We also show how you can download a fitted model for any previous run.

Make sure you have executed the [00.configuration](00.configuration.ipynb) before running this notebook.

In this notebook you would see
1. List all Experiments for the workspace
2. List AutoML runs for an Experiment
3. Get details for a AutoML Run. (Automl settings, run widget & all metrics)
4. Download fitted pipeline for any iteration


# List all AutoML Experiments in a Workspace

In [ ]:
import logging
import os
import random
import re

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd
from sklearn import datasets

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.run import Run
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun

In [ ]:
ws = Workspace.from_config()
experiment_list = Experiment.list(workspace=ws)

summary_df = pd.DataFrame(index = ['No of Runs'])
pattern = re.compile('^AutoML_[^_]*$')
for experiment in experiment_list:
    all_runs = list(experiment.get_runs())
    automl_runs = []
    for run in all_runs:
        if(pattern.match(run.id)):
            automl_runs.append(run)    
    summary_df[experiment.name] = [len(automl_runs)]
    
pd.set_option('display.max_colwidth', -1)
summary_df.T

## Diagnostics

Opt-in diagnostics for better experience, quality, and security of future releases

In [ ]:
from azureml.telemetry import set_diagnostics_collection
set_diagnostics_collection(send_diagnostics=True)

# List AutoML runs for an Experiment
You can set <i>Experiment</i> name with any experiment name from the result of the Experiment.list cell to load the AutoML runs.

In [ ]:
experiment_name = 'automl-local-classification' # Replace this with any project name from previous cell

proj = ws.experiments()[experiment_name]
summary_df = pd.DataFrame(index = ['Type', 'Status', 'Primary Metric', 'Iterations', 'Compute', 'Name'])
pattern = re.compile('^AutoML_[^_]*$')
all_runs = list(proj.get_runs(properties={'azureml.runsource': 'automl'}))
for run in all_runs:
    if(pattern.match(run.id)):
        properties = run.get_properties()
        tags = run.get_tags()
        amlsettings = eval(properties['RawAMLSettingsString'])
        if 'iterations' in tags:
            iterations = tags['iterations']
        else:
            iterations = properties['num_iterations']
        summary_df[run.id] = [amlsettings['task_type'], run.get_details()['status'], properties['primary_metric'], iterations, properties['target'], amlsettings['name']]
    
from IPython.display import HTML
projname_html = HTML("<h3>{}</h3>".format(proj.name))

from IPython.display import display
display(projname_html)
display(summary_df.T)

# Get Details for a Auto ML Run

Copy the project name and run id from the previous cell output to find more details on a particular run.

In [ ]:
run_id = 'AutoML_b7c4076b-181d-4ef4-ab9f-36bb44c1e36c'

from azureml.train.widgets import RunDetails

experiment = Experiment(ws, experiment_name)
ml_run = AutoMLRun(experiment=experiment, run_id=run_id)

summary_df = pd.DataFrame(index = ['Type', 'Status', 'Primary Metric', 'Iterations', 'Compute', 'Name', 'Start Time', 'End Time'])
properties = ml_run.get_properties()
tags = ml_run.get_tags()
status = ml_run.get_details()
amlsettings = eval(properties['RawAMLSettingsString'])
if 'iterations' in tags:
    iterations = tags['iterations']
else:
    iterations = properties['num_iterations']
start_time = None
if 'startTimeUtc' in status:
    start_time = status['startTimeUtc']
end_time = None
if 'endTimeUtc' in status:
    end_time = status['endTimeUtc']
summary_df[ml_run.id] = [amlsettings['task_type'], status['status'], properties['primary_metric'], iterations, properties['target'], amlsettings['name'], start_time, end_time]
display(HTML('<h3>Runtime Details</h3>'))
display(summary_df)

#settings_df = pd.DataFrame(data=amlsettings, index=[''])
display(HTML('<h3>AutoML Settings</h3>'))
display(amlsettings)

display(HTML('<h3>Iterations</h3>'))
RunDetails(ml_run).show() 

children = list(ml_run.get_children())
metricslist = {}
for run in children:
    properties = run.get_properties()
    metrics = {k: v for k, v in run.get_metrics().items() if isinstance(v, float)}    
    metricslist[int(properties['iteration'])] = metrics

rundata = pd.DataFrame(metricslist).sort_index(1)
display(HTML('<h3>Metrics</h3>'))
display(rundata)


# Download fitted models

## Download best model for any given metric

In [ ]:
metric = 'AUC_weighted' # Replace with a metric name
best_run, fitted_model = ml_run.get_output(metric=metric)
fitted_model

## Download model for any given iteration

In [ ]:
iteration = 4 # Replace with an interation number
best_run, fitted_model = ml_run.get_output(iteration=iteration)
fitted_model

# Register fitted model for deployment

In [ ]:
description = 'AutoML Model'
tags = None
ml_run.register_model(description=description, tags=tags)
ml_run.model_id # Use this id to deploy the model as a web service in Azure

## Register best model for any given metric

In [ ]:
metric = 'AUC_weighted' # Replace with a metric name
description = 'AutoML Model'
tags = None
ml_run.register_model(description=description, tags=tags, metric=metric)
ml_run.model_id # Use this id to deploy the model as a web service in Azure

## Register model for any given iteration

In [ ]:
iteration = 4 # Replace with an interation number
description = 'AutoML Model'
tags = None
ml_run.register_model(description=description, tags=tags, iteration=iteration)
ml_run.model_id # Use this id to deploy the model as a web service in Azure